# Find proteins outside the 8p loss event (trans) that are commonly different between samples with and without the event

# Only looking at interacting proteins

## Setup

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
CHROMOSOME = "8"
ARM = "p"
TRANS_OR_CIS = "trans"

ttest_results_file = f"{CHROMOSOME}{ARM}_{TRANS_OR_CIS}effects_filtered.tsv"

ttest_results = pd.\
read_csv(ttest_results_file, sep="\t").\
rename(columns={"Name": "protein"}).\
set_index("protein")

In [3]:
ttest_results

,luad_Database_ID,lscc_Database_ID,brca_Database_ID,ovarian_Database_ID,brca_pvalue,colon_pvalue,hnscc_pvalue,lscc_pvalue,luad_pvalue,ovarian_pvalue,brca_diff,colon_diff,hnscc_diff,lscc_diff,luad_diff,ovarian_diff
protein,,,,,,,,,,,,,,,,
AAR2,NP_001258803.1,NP_001258803.1,NP_001258803.1,NP_001258803,0.737640,0.421458,0.932035,0.739310,0.852713,0.994340,0.167240,0.119473,-0.014591,-0.148427,0.062387,0.039881
AARSD1,NP_001248363.1|NP_001129514.2|NP_079543.1,NP_001248363.1|NP_001129514.2|NP_079543.1,NP_001248363.1|NP_001129514.2|NP_079543.1,NaN,0.762032,NaN,0.883982,0.802625,0.962340,NaN,-0.221135,NaN,0.027992,-0.120512,0.026641,NaN
ABCE1,NP_001035809.1,NP_001035809.1,NP_001035809.1,NP_001035809,0.927567,0.991784,0.597754,0.739310,0.843611,0.684333,0.061418,0.003403,0.062801,0.173669,0.078355,-0.093025
ABCF3,NP_060828.2|NP_001338227.1|NP_001338229.1|NP_0...,NP_060828.2|NP_001338227.1|NP_001338229.1|NP_0...,NP_060828.2,NP_060828,0.762032,0.960899,0.176597,0.882852,0.904794,0.933733,0.166022,0.015720,0.147387,0.089872,0.045470,0.090238
ACLY,NP_001290203.1|NP_001087.2,NP_001290203.1|NP_001087.2,NP_001290203.1|NP_001087.2,NP_001087,0.762032,0.404133,0.180321,0.799074,0.518501,0.807980,0.217653,0.123752,0.177086,-0.135989,0.238336,-0.115389
ACLY,NP_001290203.1|NP_001087.2,NP_001290203.1|NP_001087.2,NP_001290203.1|NP_001087.2,NP_001290204,0.762032,0.404133,0.180321,0.799074,0.518501,0.830631,0.217653,0.123752,0.177086,-0.135989,0.238336,-0.113338
ACLY,NP_001290203.1|NP_001087.2,NP_001290203.1|NP_001087.2,NP_001290204.1|NP_942127.1,NP_001087,0.912214,0.404133,0.180321,0.799074,0.518501,0.807980,0.184336,0.123752,0.177086,-0.135989,0.238336,-0.115389
ACLY,NP_001290203.1|NP_001087.2,NP_001290203.1|NP_001087.2,NP_001290204.1|NP_942127.1,NP_001290204,0.912214,0.404133,0.180321,0.799074,0.518501,0.830631,0.184336,0.123752,0.177086,-0.135989,0.238336,-0.113338
ACP5,NP_001104504.1,NP_001104504.1,NP_001602.1,NaN,0.982193,0.593397,0.918751,0.819254,0.469922,NaN,-0.073797,-0.100217,-0.028051,0.197725,-0.463652,NaN


## Reshape the input dataframe
We want to get our table to have these columns:
- cancer_type
- protein
- Database_ID
- change
- p_value

Since some cancer types have database IDs and some don't, we'll slice out and reshape the info for each cancer type individually.

In [4]:
cancer_types = sorted(ttest_results.columns.to_series().str.split("_", n=1, expand=True)[0].unique())

long_results = pd.DataFrame()

for cancer_type in cancer_types:
    cancer_df = ttest_results.\
    loc[:, ttest_results.columns.str.startswith(cancer_type)].\
    dropna(axis="index", how="all").\
    reset_index(drop=False)
    
    # If the cancer type has database IDs, make a separate column that has them.
    # If not, create a column of NaNs (so that the tables all match)
    if f"{cancer_type}_Database_ID" in cancer_df.columns:
        cancer_df = cancer_df.rename(columns={f"{cancer_type}_Database_ID": "Database_ID"})
    else:
        cancer_df = cancer_df.assign(Database_ID=np.nan)
        
    # Rename the pvalue and diff columns to not have the cancer type
    cancer_df = cancer_df.rename(columns={
        f"{cancer_type}_pvalue": "adj_p",
        f"{cancer_type}_diff": "change"
    }).\
    assign(cancer_type=cancer_type)
    
    # Reorder the columns
    cancer_df = cancer_df[["cancer_type", "protein", "Database_ID", "adj_p", "change"]]
    
    # Append to the overall dataframe
    long_results = long_results.append(cancer_df)

# Drop duplicate rows and reset the index
long_results = long_results[~long_results.duplicated(keep=False)].\
reset_index(drop=True)

In [5]:
long_results

,cancer_type,protein,Database_ID,adj_p,change
0,brca,AAR2,NP_001258803.1,0.737640,0.167240
1,brca,AARSD1,NP_001248363.1|NP_001129514.2|NP_079543.1,0.762032,-0.221135
2,brca,ABCE1,NP_001035809.1,0.927567,0.061418
3,brca,ABCF3,NP_060828.2,0.762032,0.166022
4,brca,ACP5,NP_001602.1,0.982193,-0.073797
5,brca,ACTB,NP_001092.1,0.982193,-0.055616
6,brca,ACTBL2,NP_001017992.1,0.999486,-0.000117
7,brca,ADARB1,NP_056648.1|NP_001103.1|NP_056649.1|NP_0011537...,0.737640,0.266276
8,brca,AGPS,NP_003650.1,0.898788,-0.213237
9,brca,AHCYL1,NP_006612.2|NP_001229602.1,0.982246,0.017116


## Select the proteins with a significant change

In [6]:
prots = long_results[long_results["adj_p"] <= 0.05].reset_index(drop=True)

In [7]:
prots.groupby("cancer_type").count()[["protein"]]

,protein
cancer_type,
brca,4
colon,7
luad,8


## Find how many cancers each protein was different in

In [8]:
prots_summary = prots.groupby("protein").agg(**{
    "cancers": ("cancer_type", lambda x: x.sort_values().drop_duplicates(keep="first").tolist())
})

prots_summary = prots_summary.\
assign(
    num_cancers=prots_summary["cancers"].apply(len),
    tmp_sort=prots_summary["cancers"].apply(lambda x: "".join(x))
).\
sort_values(by=["num_cancers", "tmp_sort"], ascending=[False, True]).\
drop(columns="tmp_sort")

prots_summary

,cancers,num_cancers
protein,,
ATP6V1A,"[brca, colon, luad]",3
ATP6V1H,"[brca, colon, luad]",3
ATP6V1D,"[brca, luad]",2
PPP2R1A,"[brca, luad]",2
ATP6V1E1,"[colon, luad]",2
ATP6V1F,"[colon, luad]",2
HSPA4L,[colon],1
PPP2CA,[colon],1
RPS6KB2,[colon],1


In [9]:
prots_summary["num_cancers"].value_counts().sort_index(ascending=False)

3    2
2    4
1    5
Name: num_cancers, dtype: int64

## Save results

In [10]:
prots_summary = prots_summary.assign(
    cancers=prots_summary["cancers"].apply(lambda x: "_".join(x))
)

output_file = f"pancancer_summary_{CHROMOSOME}{ARM}_{TRANS_OR_CIS}_filtered.tsv"
prots_summary.to_csv(output_file, sep="\t")